# **IV–RV Mispricing**

The IV–RV spread refers to the gap between **implied volatility (IV)**, which reflects the market’s expectation of future variance, and **realized volatility (RV)**, which measures the actual variance observed in past returns. On average, IV tends to exceed RV because option sellers demand compensation for bearing tail risk and for taking the other side of persistent hedging flows. This difference is known as the **Variance Risk Premium (VRP)**.  

## Why Trade the IV–RV Spread?

While the VRP exists structurally, it is **time-varying** and can deviate significantly from fair value.  
- In calm markets, IV may remain elevated relative to RV, leading to overpriced options.  
- After shocks or during regime shifts, IV may lag realized outcomes and even understate true risk.  

By forecasting RV with historical and high-frequency data, traders can identify periods when the IV–RV spread is abnormally wide or inverted. Trading these dislocations through delta-hedged long or short volatility positions allows us to selectively capture the VRP while avoiding its most dangerous drawdowns.  

As options traders, we are not merely collecting the variance risk premium passively, we are exploiting **mispricings in its dynamics**, aiming for superior risk-adjusted returns by timing when IV is too rich or too cheap relative to expected realized volatility.  


The notebook is structured as follows:

1. [Read daily SPX Options chain & intraday SPX Index](#read_data)  
2. [Remove Illiquid Options](#plot_iv)
3. [Plot the Variance Risk Premium (VRP)](#plot_vrp)
4. [Forecast Realized Volatility]()
4. [Generate Trading Signals](#eee)
5. [Apply Trading Filters](#dded)
6. [Realistic Backtest with walk forward & Risk management](#plot_iv)  
7. [Conclusion](#plot_iv)  

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **1. Read SPX Options Data**

In this notebook we are going to consider the entire options chain, namely all strikes and expiries for each date in the year `2023`.

In [ ]:
file = "data/intermediate/full_spx_options_2023.parquet"

cols = [
    "strike", "underlying_last", 
    "dte", "expiry",
    "c_iv", "p_iv",
]

options = pd.read_parquet(file, columns=cols)
spot = options.groupby("date")["underlying_last"].first()

In [8]:
options

,strike,underlying_last,dte,expiry,c_iv,p_iv
date,,,,,,
2023-01-04,1000.0,3853.39,27.0,2023-01-31,NaN,1.35367
2023-01-04,1200.0,3853.39,27.0,2023-01-31,NaN,1.17498
2023-01-04,1400.0,3853.39,27.0,2023-01-31,NaN,1.02388
2023-01-04,1500.0,3853.39,27.0,2023-01-31,NaN,0.95679
2023-01-04,1600.0,3853.39,27.0,2023-01-31,NaN,0.89294
...,...,...,...,...,...,...
2023-12-29,5800.0,4772.17,0.0,2023-12-29,1.27964,2.46351
2023-12-29,5900.0,4772.17,0.0,2023-12-29,1.38397,2.68042
2023-12-29,6000.0,4772.17,0.0,2023-12-29,1.48449,2.91246
